In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sys
print(sys.byteorder)

In [ ]:
F = np.zeros((256+1,256),dtype=int)#use enough rows to wrap around.

for poly in range(256):
    F[1,poly] = 0x01
    for j in range(1,256):
        F[j+1,poly] = F[j,poly] << 1
        F[j+1,poly] = F[j+1,poly]&0xFF
        if 0x80&F[j,poly]:
            F[j+1,poly] = F[j+1,poly]^poly 
            
plt.figure(figsize=(10,10))
plt.rcParams['figure.figsize'] = [10, 10]
plt.imshow(F)
plt.xlabel("Generating Polynomial")
plt.ylabel("Element Index")
plt.title("Columns Represent Field Generated by Polynomial")
plt.show()


In [ ]:
#Used D-Wave's Max Cut Problem as a starting template
#https://github.com/dwave-examples/maximum-cut/blob/master/maximum_cut.py
from collections import defaultdict

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
#import networkx as nx

# Initialize our Q matrix
Q = defaultdict(int)

n = 30 #number of nodes
p = 0.15 #density parameter

for i in range(n):
    u = np.random.random()
    if u < p:
        Q[(i,i)]+= -1 #degree of node i
        Q[(i,j)]+= 1 #edge (i,j)
        Q[(j,i)]+= 1 #symmetric matix

Qmatrix = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        Qmatrix[i,j] = Q[(i,j)]

plt.figure(figsize=(10,10))
plt.rcParams['figure.figsize'] = [10, 10]
plt.imshow(Q)
plt.xlabel("Index")
plt.ylabel("Index")
plt.title("Q Matrix")
plt.show()




In [ ]:
chainstrength = 8
numruns = 10
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               label='Example - Maximum Cut')

# ------- Print results to user -------
print('-' * 60)
print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
print('-' * 60)
for sample, E in response.data(fields=['sample','energy']):
    S0 = [k for k,v in sample.items() if v == 0]
    S1 = [k for k,v in sample.items() if v == 1]
    print('{:>15s}{:>15s}{:^15s}{:^15s}'.format(str(S0),str(S1),str(E),str(int(-1*E))))

# ------- Display results to user -------
# Grab best result
# Note: "best" result is the result with the lowest energy
# Note2: the look up table (lut) is a dictionary, where the key is the node index
#   and the value is the set label. For example, lut[5] = 1, indicates that
#   node 5 is in set 1 (S1).
lut = response.first.sample

# Interpret best result in terms of nodes and edges
S0 = [node for node in G.nodes if not lut[node]]
S1 = [node for node in G.nodes if lut[node]]
cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

print(response)